# 2.cnn - pytroch - Dog Cat Classifier.ipynb

## if using google colab

### Connect Colab to Google drive 

In [ ]:
# from google.colab import drive
# drive.mount._DEBUG = True
# # drive.mount('/content/drive', force_remount=True)
# drive.mount('/content/drive')

# #Change the Directry to your project in GOogle driver (Create a folder named : "App")
# %cd "/content/drive/My Drive/Colab Notebooks"

# # List driver
# !ls


### Download data set from kaggle
[Downloading Kaggle Datasets into Google Colab](https://medium.com/@opalkabert/downloading-kaggle-datasets-into-google-colab-fb9654c94235)

In [ ]:


# !pip install -U -q kaggle
# !mkdir -p ~/.kaggle

# from google.colab import files
# # files.upload()

# !cp kaggle.json ~/.kaggle/
# # !kaggle datasets list

# !kaggle datasets download -d chetankv/dogs-cats-images
# !ls

### Downloaded data set unzip to dir

### Check GPU availability

In [ ]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import datasets, transforms, models

torch.set_printoptions(linewidth=120)  # Displa optin for output
torch.set_grad_enabled(True) # Already on by default

batch_size = 32

In [ ]:
print(torch.__version__)
print(torchvision.__version__)

In [ ]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

### Preprocess images

In [ ]:
# Kaggle
data_dir = '../input/dog vs cat/dataset'
# colab path
# data_dir = 'dog vs cat'
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

# test_transform = transforms.Compose([
#                                 transforms.Resize(254),
#                                 transforms.CenterCrop(254),
#                                 transforms.ToTensor()])


train_data =  torchvision.datasets.ImageFolder(data_dir + '/training_set', transform = transform)
test_data =  torchvision.datasets.ImageFolder(data_dir + '/test_set', transform = transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
images, labels = next(iter(train_loader))

In [ ]:
images, labels = next(iter(train_loader))
print(' images shape', images.shape)
print(' labels shape', labels.shape)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# helper function to un-normalize and display an image
def imshow(img):
    img = img 
    convimg = np.transpose(img, (1, 2, 0))
    print('after change np.transpose', convimg.shape)
    plt.imshow(convimg, cmap='gray',interpolation='none')  # convert from Tensor image
    
print('before ', images[0].shape)    
imshow(images[0])

In [ ]:
imshow(images[1])


In [ ]:

imshow(images[2])


In [ ]:
imshow(images[3])

In [ ]:

imshow(images[4])

In [ ]:

imshow(images[5])

In [ ]:

imshow(images[6])

In [ ]:

imshow(images[7])


In [ ]:
imshow(images[8])


In [ ]:
imshow(images[9])


In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # convolutional layer
        self.conv1 = nn.Conv2d(3, 16, 5)
        # max pooling layer
        self.maxpool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(32*53*53, 256)
        self.fc2 = nn.Linear(256, 84)
        self.fc3 = nn.Linear(84, 2)
        self.softmax = nn.LogSoftmax(dim=1)

        
    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        print('x 1 raw:', x.shape)
        x = self.conv1(x)
        print('x 1 after conv1:', x.shape)
        x = F.relu(x)
        print('x 1 after relue:', x.shape)
        x = self.maxpool(x)
        print('x 1 after max fool:', x.shape)
        print(' end conv 1-------------------------------------------------------------------------------- ')
        
#       x = self.pool(F.relu(self.conv2(x)))
        print('x 2 raw:', x.shape)
        x = self.conv2(x)
        print('x 2 after conv2:', x.shape)
        x = F.relu(x)
        print('x 2 after relue:', x.shape)
        x = self.maxpool(x)
        print('x 2 after max fool:', x.shape)
        
        print(' end conv 2-------------------------------------------------------------------------------- ')
        
        x = self.dropout(x)
        print('x 3 dropout', x.shape)      

        x = x.view(-1, 32 * 53 * 53)
        print('x 4 after flatter', x.shape)
        
        print(' x = x.view(-1, 32 * 53 * 53) end flattern -------------------------------------------------------------------------------- ')

#       x = F.relu(self.fc1(x))        
        x = self.fc1(x)
        print('x 5 after fc1:', x.shape)
        x = F.relu(x)
        print('x 5 after relue:', x.shape)
        
        print('  fc1 end -------------------------------------------------------------------------------- ')
        
#       x = self.dropout(F.relu(self.fc2(x)))        
        x = self.fc2(x)
        print('x 6 after fc1:', x.shape)
        x = F.relu(x)
        print('x 6 after relu:', x.shape)
        x = self.dropout(x)
        print('x 6 dropout', x.shape)
        
        print('  fc2 end -------------------------------------------------------------------------------- ')

#       x = self.softmax(self.fc3(x))
        x = self.fc3(x)
        print('x 7 after fc3', x.shape)
        x = self.softmax(x)
        print('x 6 after softmax:', x.shape)
        
        return x
            

In [ ]:
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [ ]:
epochs = 10
steps = 0
train_losses, test_losses = [], []
for e in range(epochs):
    print('Start epoch', e, '=======================================')
    running_loss = 0
    for images, labels in train_loader:
        logps = model(images)
        loss = criterion(logps, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        
        test_loss = 0
        accuracy = 0
#         # Trun off gradionts for validation, saves memeory and computations
#         with torch.no_grad():
#             model.eval()
#             for images, labels in test_loader:
#                 log_ps = model(images)
#                 test_loss += criterion(log_ps, labels)
                
#                 ps =  torch.exp(log_ps)
#                 top_p, top_class = ps.topk(1, dim=1)
#                 equals = top_class == labels.view(*top_class.shape)
#                 accuracy += torch.mean(equals.type(torch.FloatTensor))
                
        model.train()
        train_losses.append(running_loss/len(train_loader))
        test_losses.append(test_loss/len(test_loader))
        
        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(test_loader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(test_loader))
             )

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='validation loss')
plt.legend(frameon=False)